In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split

# Loading
import numpy as np
from keras.utils import np_utils


# Load the training set
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = np.asarray([mpimg.imread(image_dir + files[i]) for i in range(n)])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = np.asarray([mpimg.imread(gt_dir + files[i]) for i in range(n)])


x_train, x_test, y_train, y_test = train_test_split(imgs, gt_imgs, test_size=0.2)
print(x_train.shape, y_train.shape)

# input image dimensions
img_dim = 400
div = 1
if img_dim % div != 0:
    print("Invalid divider for the image dimensions!")
img_rows, img_cols = img_dim//div, img_dim//div

x_train = x_train.reshape(x_train.shape[0]*div*div, img_rows, img_cols, 3)
x_test = x_test.reshape(x_test.shape[0]*div*div, img_rows, img_cols, 3)
    
y_train = y_train.reshape(y_train.shape[0]*div*div, img_rows, img_cols)
y_test = y_test.reshape(y_test.shape[0]*div*div, img_rows, img_cols)

input_shape = (img_rows, img_cols, 3)
    
print(x_train.shape, y_train.shape)

batch_size = 128
num_classes = 2
epochs = 12

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

window_size = 30
patch_size = 16
input_shape = (window_size, window_size, 3)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
print(y_train.shape)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

def generate_minibatch(X, Y):
    """
    Procedure for real-time minibatch creation and image augmentation.
    This runs in a parallel thread while the model is being trained.
    """
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, window_size, window_size, 3))
        Y_batch = np.empty((batch_size, 2))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
            shape = X[idx].shape

            # Sample a random window from the image
            center = np.random.randint(window_size//2, shape[0] - window_size//2, 2)
            sub_image = X[idx][center[0]-window_size//2:center[0]+window_size//2,
                               center[1]-window_size//2:center[1]+window_size//2]
            gt_sub_image = Y[idx][center[0]-patch_size//2:center[0]+patch_size//2,
                                  center[1]-patch_size//2:center[1]+patch_size//2]

            # The label does not depend on the image rotation/flip (provided that the rotation is in steps of 90°)
            threshold = 0.25
            label = (np.array([np.mean(gt_sub_image)]) > threshold) * 1

            label = np_utils.to_categorical(label, num_classes)
            X_batch[i] = sub_image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

Loading 100 images
Loading 100 images
(80, 400, 400, 3) (80, 400, 400)
(80, 400, 400, 3) (80, 400, 400)
x_train shape: (80, 400, 400, 3)
80 train samples
20 test samples
(80, 400, 400, 2)


In [3]:
model.fit_generator(generate_minibatch(x_train, y_train),
                    steps_per_epoch=5,
                    epochs=epochs,
                    verbose=1)

Epoch 1/12
5/5 [==============================] - 2s 436ms/step - loss: 0.6791 - acc: 0.8438
Epoch 2/12
5/5 [==============================] - 2s 395ms/step - loss: 0.4074 - acc: 1.0000
Epoch 3/12
5/5 [==============================] - 2s 341ms/step - loss: 0.0318 - acc: 1.0000
Epoch 4/12
5/5 [==============================] - 2s 364ms/step - loss: 0.0072 - acc: 1.0000
Epoch 5/12
5/5 [==============================] - 2s 390ms/step - loss: 0.0037 - acc: 1.0000
Epoch 6/12
5/5 [==============================] - 2s 372ms/step - loss: 0.0026 - acc: 1.0000
Epoch 7/12
5/5 [==============================] - 2s 365ms/step - loss: 0.0017 - acc: 1.0000
Epoch 8/12
5/5 [==============================] - 2s 339ms/step - loss: 0.0012 - acc: 1.0000
Epoch 9/12
5/5 [==============================] - 2s 347ms/step - loss: 0.0011 - acc: 1.0000
Epoch 10/12
5/5 [==============================] - 2s 379ms/step - loss: 9.1527e-04 - acc: 1.0000
Epoch 11/12
5/5 [==============================] - 2s 362ms/step 

In [4]:
import itertools

xy_test_window = list(itertools.islice(generate_minibatch(x_test, y_test), 10))
# TODO real testing and prediction of the patches
xxx = list(zip(*xy_test_window))
score = model.evaluate(xxx[0][0], xxx[1][0], verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.000224461462494
Test accuracy: 1.0
